# This is an example notebook on how to use NAM for shape matching
In this notebook we consider different refinement with the spectral embeddings


In [ ]:
from geomfum.shape.mesh import TriangleMesh
from geomfum.refine import ZoomOut

import torch
import numpy as np

import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from model.neural_adjoint_map import NeuralAdjointMap
from sklearn.neighbors import NearestNeighbors

from model.neural_zoomout import NeuralZoomOut


In [ ]:
import os
from urllib.request import urlretrieve

faust_url = "https://raw.githubusercontent.com/JM-data/PyFuncMap/4bde4484c3e93bff925a6a82da29fa79d6862f4b/FAUST_shapes_off/"
shape_files = ["tr_reg_080.off", "tr_reg_093.off"]
for fname in shape_files:
    url = faust_url + fname
    out_path = os.path.join("../data/", fname)
    urlretrieve(url, out_path)


In [ ]:
mesh1 = TriangleMesh.from_file("../data/tr_reg_080.off")
mesh2 = TriangleMesh.from_file("../data/tr_reg_093.off")

eigvals1, eigvecs1 = mesh1.laplacian.find_spectrum(spectrum_size=200)
eigvals2, eigvecs2 = mesh2.laplacian.find_spectrum(spectrum_size=200)

In [ ]:
p2p_gt = np.arange(mesh1.n_vertices)

# Initial Map

In [ ]:
k = 20
C21 = np.linalg.pinv(eigvecs1[:, :k]) @ eigvecs2[p2p_gt, :20]

A12 = C21.T
# Fit KNN on eigvecs2
knn = NearestNeighbors(n_neighbors=1, algorithm="auto").fit(eigvecs2[:, :k] @ A12)
distances, indices = knn.kneighbors(eigvecs1[:, :20])

p2p_ini = indices.flatten()

print("Conversion Error:", np.mean((mesh2.vertices[p2p_ini] - mesh2.vertices) ** 2))

# ZoomOut

In [ ]:
zoomout = ZoomOut(nit=18, step=10)
ref_fmap = zoomout(C21, mesh1.basis, mesh2.basis)

In [ ]:
# Fit KNN on eigvecs2
knn = NearestNeighbors(n_neighbors=1, algorithm="auto").fit(eigvecs2 @ ref_fmap)
distances, indices = knn.kneighbors(eigvecs1)

p2p = indices.flatten()

print("Conversion Error:", np.mean((mesh2.vertices[p2p] - mesh2.vertices) ** 2))

# NeuralZoomOut

In [ ]:
nam = NeuralAdjointMap(
    torch.tensor(eigvecs1[:, :20]).to(torch.float32),
    torch.tensor(eigvecs2[:, :20]).to(torch.float32),
)

nam.optimize_from_p2p(p2p_ini)


In [ ]:
nzo = NeuralZoomOut(nit=2, step=20)
nam = nzo(
    nam,
    torch.tensor(eigvecs1).to(torch.float32),
    torch.tensor(eigvecs2).to(torch.float32),
)

emb2_nn = nam(
    torch.tensor(eigvecs2[:, :60]).to(torch.float32),
)


In [ ]:
knn = NearestNeighbors(n_neighbors=1, algorithm="auto").fit(
    emb2_nn.detach().cpu().numpy()
)
distances, indices = knn.kneighbors(eigvecs1[:, :40])

p2p = indices.flatten()

In [ ]:
print("Conversion Error:", np.mean((mesh2.vertices[p2p] - mesh2.vertices) ** 2))